# Import necessary libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Read data using pandas

In [3]:
# openpyxl must be installed.
df = pd.read_excel("sales.xlsx")
df.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### shape and columns

In [8]:
df.shape

(525461, 8)

In [5]:
df.columns

Index(['InvoiceNumber', 'ProductCode', 'ProductName', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerId', 'Country'],
      dtype='object')

<h2 align=left style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="green">
Preprocessing data
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Removing rows which is not including <code>CustomerId</code>
</font>
</p>

In [4]:
df_no_missing_customer = df.copy()
df_no_missing_customer.dropna(subset=["CustomerId"], inplace=True)
df_no_missing_customer.isna().sum()

InvoiceNumber    0
ProductCode      0
ProductName      0
Quantity         0
InvoiceDate      0
UnitPrice        0
CustomerId       0
Country          0
dtype: int64

<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Delete duplicate data
</font>
</p>

In [5]:
df_no_duplicate = df_no_missing_customer.copy()
df_no_duplicate.drop_duplicates(inplace=True)

<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Delete rows which have 0 or negative value for <code>UnitPrice</code>
</font>
</p>

In [6]:
df_no_missing_price = df_no_duplicate.copy()
df_no_missing_price = df_no_missing_price[df_no_missing_price["UnitPrice"] > 0]
df_no_missing_price.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Percentage of canceled orders
</font>
</p>

In [8]:
canceled_orders_count = df_no_missing_price[df_no_missing_price["InvoiceNumber"].str.startswith("C", na=False)]["InvoiceNumber"].nunique()
all_orders = df_no_missing_price["InvoiceNumber"].nunique()
percentage_canceled_orders = round(canceled_orders_count / all_orders * 100)
percentage_canceled_orders

19

<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Top 5 customers with must canceled order.
</font>
</p>

In [21]:
df_temp = df_no_missing_price.copy()
df_temp["CustomerId"] = df_temp["CustomerId"].apply(lambda x : str(int(x)))

group_canceled_customer_id = df_temp[df_temp["InvoiceNumber"].str.startswith("C", na=False)].groupby("CustomerId")

top_5_canceled_customers = group_canceled_customer_id["InvoiceNumber"].nunique().reset_index(name= "count").sort_values(by=["count"], ascending=False).head(5)
top_5_canceled_customers

,CustomerId,count
845,14911,65
720,14527,39
963,15311,37
599,14156,36
1693,17841,35


<p dir=rtl style="direction: rtl;text-align: left;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
Remove canceled orders
</font>
</p>

In [23]:
df_no_canceled_invoice = df_no_missing_price.copy()

df_no_canceled_invoice = df_no_canceled_invoice[~df_no_canceled_invoice["InvoiceNumber"].str.startswith("C", na=False)]

df_no_canceled_invoice.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Save preprocessed data to new file

In [24]:
df_no_canceled_invoice.to_csv("preprocessed_sales.csv", index=False)